In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from binance.exceptions import BinanceAPIException

import numpy as np
import pandas as pd
import json
import time
import schedule

In [2]:
client = Client('   ', '  ')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
quote_ccy = 'USDT'

In [6]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 500]

In [9]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
def is_candidate(filtered_klines_info, tolerable_volume):
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    
    trigger_kline = filtered_klines_info[-1]
    
    for context_kline in context_klines_info:
        
        volume_gate = float(context_kline[3])  <  tolerable_volume * 2.5
        if volume_gate :
            is_candidate = True
        else:
            return False


    trigger_gate = float(trigger_kline[3]  > tolerable_volume * 10) and (trigger_kline[-1] < 9 ) and (trigger_kline[-1] > 3 ) 


    if trigger_gate:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [11]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [12]:
def job(): 
    
    try:
        
        for target_ticker in target_ticker_list:
            print('Analyzing Hourly Started ')
            print(target_ticker)
            klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_15MINUTE, "1 day ago UTC") 
            length = int(len(klines))
            if(length < 96):
                break

            array = np.array(klines, dtype=np.float64)
            focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)
            percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
            percentage_value = np.reshape(percentage_value, (length, 1))
            focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
            average_volume = np.mean(focused_kline_array[:,3])
            tolerable_volume = average_volume * 1.2

            '''filtered_klines_info = focused_kline_array[x:x+24] '''
            filtered_klines_info = focused_kline_array
            buy_kline_info = filtered_klines_info[-1]
            if(len(filtered_klines_info) == 96 and is_candidate(filtered_klines_info, tolerable_volume)):

                print('********** SUCCESS ************************')
                print(target_ticker)
                print('Buy Price')
                print(buy_kline_info[2])

                busd = 10

                quantity = busd/ buy_kline_info[2]
                print('Quantity')
                print(quantity)
                quantity_str = "{:0.0{}f}".format(quantity, 2)
                print(quantity_str)

                order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
                time.sleep(5)
                order_id = order['orderId']
                buy_price = order['fills'][0]['price']
                buy_qty = order['executedQty']
                sell_price = buy_price * 1.03



                while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                    time.sleep(10)
                
                if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                    asset = target_ticker.split("USDT")[0]
                    balance = client.get_asset_balance(asset=asset)
                    free_to_trade = balance['free']
                    sell_price_str = "{:0.0{}f}".format(sell_price, 2)
                
                    order = client.create_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                    print('********** SELL ORDER SET ************************')
        print('**** Hourly Analyze Done ******')
    except BinanceAPIException as e:
        print(e.status_code)
        print(e.message)

In [13]:
'''schedule.every(15).minutes.do(job) '''

'schedule.every(15).minutes.do(job) '

In [14]:
schedule.every().hour.at(":13").do(job)

Every 1 hour at 00:13:00 do job() (last run: [never], next run: 2022-07-12 13:13:00)

In [15]:
schedule.every().hour.at(":28").do(job)

Every 1 hour at 00:28:00 do job() (last run: [never], next run: 2022-07-12 12:28:00)

In [16]:
schedule.every().hour.at(":43").do(job)

Every 1 hour at 00:43:00 do job() (last run: [never], next run: 2022-07-12 12:43:00)

In [17]:
schedule.every().hour.at(":58").do(job)

Every 1 hour at 00:58:00 do job() (last run: [never], next run: 2022-07-12 12:58:00)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

Analyzing Hourly Started 
FETUSDT
Analyzing Hourly Started 
DUSKUSDT
Analyzing Hourly Started 
COCOSUSDT
Analyzing Hourly Started 
TOMOUSDT
Analyzing Hourly Started 
MFTUSDT
Analyzing Hourly Started 
KEYUSDT
Analyzing Hourly Started 
WANUSDT
Analyzing Hourly Started 
BANDUSDT
Analyzing Hourly Started 
NKNUSDT
Analyzing Hourly Started 
ARPAUSDT
Analyzing Hourly Started 
TROYUSDT
Analyzing Hourly Started 
BTSUSDT
Analyzing Hourly Started 
LTOUSDT
Analyzing Hourly Started 
MBLUSDT
Analyzing Hourly Started 
VTHOUSDT
Analyzing Hourly Started 
BLZUSDT
Analyzing Hourly Started 
ANTUSDT
Analyzing Hourly Started 
YFIIUSDT
Analyzing Hourly Started 
DIAUSDT
Analyzing Hourly Started 
BELUSDT
Analyzing Hourly Started 
SUNUSDT
Analyzing Hourly Started 
FLMUSDT
Analyzing Hourly Started 
ORNUSDT
Analyzing Hourly Started 
UTKUSDT
Analyzing Hourly Started 
XVSUSDT
Analyzing Hourly Started 
ALPHAUSDT
Analyzing Hourly Started 
CTKUSDT
Analyzing Hourly Started 
DNTUSDT
Analyzing Hourly Started 
UNFIUSDT
An